In [1]:
# Import necessary libraries for data manipulation, machine learning model building, and memory management
import os
import random
import numpy as np
import pandas as pd
import tqdm
import multiprocessing

In [2]:
# Constants for defining the range of overlaps as a string
NUMBER_OF_OVERLAPS = "1-12"

# Determine the current working directory
current_directory = os.getcwd()
# Define the working directory as one level up from the current directory
WORKING_DIR = os.path.join(current_directory, "..")

# Path configurations for data storage and model saving
PATH_DATA = os.path.join(WORKING_DIR, "Data") 
PATH_TRAINING_DATA = os.path.join(PATH_DATA, "3_data_npy_train")
PATH_TRAINING_DATA = os.path.join(PATH_TRAINING_DATA, NUMBER_OF_OVERLAPS)
PATH_MODELS_PARENT = os.path.join(PATH_DATA, "models")

# Ensure the model directory exists, create it if not
os.makedirs(PATH_MODELS_PARENT, exist_ok=True)


In [3]:
# Loading Filelist of Training-data
filelist = os.listdir(PATH_TRAINING_DATA)

filelist = [(x,y) for x in filelist if '_x_' in x for y in filelist if x.split('_')[0] == y.split('_')[0] and "_y_" in y]
filelist.sort()

In [5]:
def load_data(count):

    files = random.sample(filelist, k=count)

    x_lst = []
    y_lst = []

    for file in tqdm.tqdm(files):
        x_tmp = np.load(PATH_TRAINING_DATA + "/" + file[0])
        y_tmp = np.load(PATH_TRAINING_DATA+ "/" + file[1])

        if INPUT_SIZE > x_tmp.shape[1]:
            raise UserWarning("Length to height")

        x_tmp = x_tmp[:,:INPUT_SIZE]

        x_lst.append(x_tmp)
        y_lst.append(y_tmp)

    x = np.concatenate(x_lst, axis=0)
    y = np.concatenate(y_lst, axis=0)

    print("calculating float32")
    x = np.subtract(x ,65)
    x = np.array(x, dtype='float32')
    print (x)
    print (y)
    x = np.divide(x , 25)

    y = np.array(y, dtype='float32')

    return x, y

In [6]:
def load_partial_data(count, records_per_file):
    """
    Load a specific number of records from a set number of files.

    Parameters:
    - count: The number of files to randomly select and load data from.
    - records_per_file: The number of records to load from each file.

    Returns:
    - Tuple of np.arrays: The loaded x and y data.
    """

    # Assuming filelist is a list of tuples/lists with paths for x and y data files
    files = random.sample(filelist, k=count)

    x_lst = []
    y_lst = []

    for file in tqdm.tqdm(files):
        # Load only the specified number of records from each file
        x_tmp = np.load(PATH_TRAINING_DATA + "/" + file[0])[:records_per_file]
        y_tmp = np.load(PATH_TRAINING_DATA  + "/" + file[1])[:records_per_file]

        # Ensure the input size matches expected dimensions, adjust if necessary
        if INPUT_SIZE > x_tmp.shape[1]:
            raise UserWarning("Input size too large for loaded data.")

        x_tmp = x_tmp[:, :INPUT_SIZE] # Trim or expand the x data to the INPUT_SIZE


        x_lst.append(x_tmp)
        y_lst.append(y_tmp)
        
    # Concatenate all loaded data into a single array for both x and y
    x = np.concatenate(x_lst, axis=0)
    y = np.concatenate(y_lst, axis=0)

    # Preprocess the data as before
    print("Calculating float32...")
    x = np.subtract(x, 65)
    x = np.array(x, dtype='float32')
    x = np.divide(x, 25)
    
    # Convert y data to float32
    y = np.array(y, dtype='float32')

    return x, y

In [7]:
from tensorflow import keras
from tensorflow.keras import layers

In [11]:
# Define your ResNet-like architecture
def make_resnet(input_length, num_filters=32, num_outputs=1, d1=512, d2=512, ks=5, depth=5, reg_param=0.0002, final_activation='sigmoid'):
    inp = layers.Input(shape=(input_length, 1))
    
    # First convolutional layer
    conv0 = layers.Conv1D(num_filters, kernel_size=1, padding='same', kernel_regularizer=l2(reg_param))(inp)
    conv0 = layers.BatchNormalization()(conv0)
    conv0 = layers.Activation('relu')(conv0)

    # Residual blocks
    shortcut = conv0
    for i in range(depth):
        conv1 = layers.Conv1D(num_filters, kernel_size=ks, padding='same', kernel_regularizer=l2(reg_param))(shortcut)
        conv1 = layers.BatchNormalization()(conv1)
        conv1 = layers.Activation('relu')(conv1)
        conv2 = layers.Conv1D(num_filters, kernel_size=ks, padding='same', kernel_regularizer=l2(reg_param))(conv1)
        conv2 = layers.BatchNormalization()(conv2)
        conv2 = layers.Activation('relu')(conv2)
        shortcut = layers.Add()([shortcut, conv2])

    # Output layers
    flat1 = layers.Flatten()(shortcut)
    dense1 = layers.Dense(d1, kernel_regularizer=l2(reg_param))(flat1)
    dense1 = layers.BatchNormalization()(dense1)
    dense1 = layers.Activation('relu')(dense1)
    dense2 = layers.Dense(d2, kernel_regularizer=l2(reg_param))(dense1)
    dense2 = layers.BatchNormalization()(dense2)
    dense2 = layers.Activation('relu')(dense2)
    out = layers.Dense(num_outputs, activation=final_activation, kernel_regularizer=l2(reg_param))(dense2)

    model = models.Model(inputs=inp, outputs=out)
    return model

In [ ]:
# Import necessary libraries for data manipulation, machine learning model building, and memory management

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, callbacks, optimizers
from tensorflow.keras.regularizers import l2
import gc
import os
# Define input size for the model and the path where models will be saved
INPUT_SIZE = 200
PATH_MODELS =os.path.join(PATH_MODELS_PARENT,f"models_seq_{INPUT_SIZE}")
os.makedirs(PATH_MODELS, exist_ok=True)

#pinwheel_sizes = [26, 25, 23, 21, 19, 17]
pinwheel_sizes = [26]

# Cumulative sizes used for iterating over segments
cumulative_sizes = np.cumsum([0] + pinwheel_sizes)


model_accuracies = []

# Loop over each sequence length
seq_len = INPUT_SIZE  # Define sequence length
# Iterate over each wheel
for wheel, (start_pin, end_pin) in enumerate(zip(cumulative_sizes, cumulative_sizes[1:])):
    train_all_pins = False  # Flag to determine if all pins should be trained

    # Iterate over each pin in the current wheel
    for pin in range(start_pin, end_pin):
        while True:
            # Extract the target values for the current pin
            x, y = load_partial_data(100,15000)

            targets = y[:, pin]
            # Split the data into training and testing sets
            X_train, X_test, y_train, y_test = train_test_split(x, targets, test_size=0.2, random_state=17)

            # Create and compile the ResNet model
            model_pin = make_resnet(seq_len, num_filters=100, depth=5, num_outputs=1)
            model_pin.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])

            # Setup callbacks for model saving and learning rate scheduling
            checkpoint_cb_pin = callbacks.ModelCheckpoint(PATH_MODELS+f'/best_model_wheel_{wheel}_pin_{pin}.h5', save_best_only=True)
            lr_scheduler_cb_pin = callbacks.LearningRateScheduler(lambda epoch: 1e-3 * 0.95 ** epoch)

            # Train the model
            history_pin = model_pin.fit(X_train, y_train, batch_size=1000, epochs=10, validation_split=0.2, callbacks=[checkpoint_cb_pin, lr_scheduler_cb_pin])
            # Evaluate the model on the test set
            test_loss_pin, test_accuracy_pin = model_pin.evaluate(X_test, y_test)
            print(f"Seq Length {seq_len}, Wheel {wheel}, Pin {pin}: Test Loss: {test_loss_pin}, Test Accuracy: {test_accuracy_pin}")

            # Check if the current pin is the first pin
            if pin == start_pin:
                # If the accuracy is higher than 0.6, set the flag to train all pins
                if test_accuracy_pin > 0.5:
                    train_all_pins = True
                else:
                    break  # Skip training the remaining pins for this wheel
            #retraining for pins vs low accuracy        
            if test_accuracy_pin > 0.88:

                # Record the model's accuracy if it's the first pin or if all pins are being trained
                if train_all_pins or pin == start_pin:
                    model_accuracies.append({
                        'Sequence Length': seq_len,
                        'Wheel Number': wheel,
                        'Pin Number': pin,
                        'Accuracy': test_accuracy_pin
                    })

                    # Save the final model
                    model_pin.save(PATH_MODELS+f'/best_model_wheel_{wheel}_pin_{pin}.h5')

                # Clear the TensorFlow session and collect garbage to free memory
                tf.keras.backend.clear_session()
                gc.collect()
                break

            
# Convert the list of accuracies to a DataFrame
accuracy_df = pd.DataFrame(model_accuracies)

# Save the DataFrame to an Excel file
accuracy_df.to_excel('model_accuracies.xlsx', index=False)

In [22]:
# Same code as above but for different sequence length
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, callbacks, optimizers
from tensorflow.keras.regularizers import l2
import gc
import os

#Sequence lenght
INPUT_SIZE = 104
PATH_MODELS =os.path.join(PATH_MODELS_PARENT,f"models_seq_{INPUT_SIZE}")
os.makedirs(PATH_MODELS, exist_ok=True)

pinwheel_sizes = [26]

cumulative_sizes = np.cumsum([0] + pinwheel_sizes)


model_accuracies = []

# Loop over each sequence length
seq_len = INPUT_SIZE
# Iterate over each wheel
for wheel, (start_pin, end_pin) in enumerate(zip(cumulative_sizes, cumulative_sizes[1:])):
    train_all_pins = False  # Flag to determine if all pins should be trained

    # Iterate over each pin in the current wheel
    for pin in range(5, end_pin):
        while True:
            # Extract the target values for the current pin
            x, y = load_partial_data(10,1500)

            targets = y[:, pin]
            # Split the data into training and testing sets
            X_train, X_test, y_train, y_test = train_test_split(x, targets, test_size=0.2, random_state=17)

            # Create and compile the ResNet model
            model_pin = make_resnet(seq_len, num_filters=26, depth=5, num_outputs=1)
            model_pin.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])

            # Setup callbacks for model saving and learning rate scheduling
            checkpoint_cb_pin = callbacks.ModelCheckpoint(PATH_MODELS+f'/best_model_wheel_{wheel}_pin_{pin}.h5', save_best_only=True)
            lr_scheduler_cb_pin = callbacks.LearningRateScheduler(lambda epoch: 1e-3 * 0.95 ** epoch)

            # Train the model
            history_pin = model_pin.fit(X_train, y_train, batch_size=260, epochs=10, validation_split=0.2, callbacks=[checkpoint_cb_pin, lr_scheduler_cb_pin])
            # Evaluate the model on the test set
            test_loss_pin, test_accuracy_pin = model_pin.evaluate(X_test, y_test)
            print(f"Seq Length {seq_len}, Wheel {wheel}, Pin {pin}: Test Loss: {test_loss_pin}, Test Accuracy: {test_accuracy_pin}")

            # Check if the current pin is the first pin
            if pin == start_pin:
                # If the accuracy is higher than 0.6, set the flag to train all pins
                if test_accuracy_pin > 0.5:
                    train_all_pins = True
                else:
                    break  # Skip training the remaining pins for this wheel
            #retraining for pins vs low accuracy        
            if test_accuracy_pin > 0.62:

                # Record the model's accuracy if it's the first pin or if all pins are being trained
                if train_all_pins or pin == start_pin:
                    model_accuracies.append({
                        'Sequence Length': seq_len,
                        'Wheel Number': wheel,
                        'Pin Number': pin,
                        'Accuracy': test_accuracy_pin
                    })

                    # Save the final model
                    model_pin.save(PATH_MODELS+f'/best_model_wheel_{wheel}_pin_{pin}.h5')

                # Clear the TensorFlow session and collect garbage to free memory
                tf.keras.backend.clear_session()
                gc.collect()
                break

            
# Convert the list of accuracies to a DataFrame
accuracy_df = pd.DataFrame(model_accuracies)

# Save the DataFrame to an Excel file
accuracy_df.to_excel('model_accuracies.xlsx', index=False)

100%|██████████| 100/100 [00:02<00:00, 49.50it/s]


Calculating float32...
Epoch 1/10
1847/1847 [==============================] - 59s 30ms/step - loss: 0.7944 - accuracy: 0.5946 - val_loss: 0.7042 - val_accuracy: 0.6004


/home/cryptool/anaconda3/envs/tensorflow/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/10
1847/1847 [==============================] - 53s 29ms/step - loss: 0.6819 - accuracy: 0.6109 - val_loss: 0.6805 - val_accuracy: 0.6054
Epoch 3/10
1847/1847 [==============================] - 54s 29ms/step - loss: 0.6690 - accuracy: 0.6142 - val_loss: 0.6713 - val_accuracy: 0.6125
Epoch 4/10
1847/1847 [==============================] - 54s 29ms/step - loss: 0.6599 - accuracy: 0.6187 - val_loss: 0.6744 - val_accuracy: 0.6073
Epoch 5/10
1847/1847 [==============================] - 54s 29ms/step - loss: 0.6512 - accuracy: 0.6223 - val_loss: 0.6539 - val_accuracy: 0.6187
Epoch 6/10
1847/1847 [==============================] - 54s 29ms/step - loss: 0.6450 - accuracy: 0.6248 - val_loss: 0.6501 - val_accuracy: 0.6204
Epoch 7/10
1847/1847 [==============================] - 54s 29ms/step - loss: 0.6405 - accuracy: 0.6266 - val_loss: 0.6465 - val_accuracy: 0.6230
Epoch 8/10
1847/1847 [==============================] - 54s 29ms/step - loss: 0.6369 - accuracy: 0.6285 - val_loss: 0.6436 -

100%|██████████| 100/100 [00:01<00:00, 50.29it/s]


...

Calculating float32...
Epoch 1/10
1847/1847 [==============================] - 58s 30ms/step - loss: 0.7686 - accuracy: 0.6376 - val_loss: 0.6756 - val_accuracy: 0.6441
Epoch 2/10
1847/1847 [==============================] - 55s 30ms/step - loss: 0.6508 - accuracy: 0.6557 - val_loss: 0.6509 - val_accuracy: 0.6521
Epoch 3/10
1847/1847 [==============================] - 56s 30ms/step - loss: 0.6345 - accuracy: 0.6603 - val_loss: 0.6429 - val_accuracy: 0.6524
Epoch 4/10
1847/1847 [==============================] - 54s 29ms/step - loss: 0.6212 - accuracy: 0.6658 - val_loss: 0.6363 - val_accuracy: 0.6568
Epoch 5/10
1847/1847 [==============================] - 54s 29ms/step - loss: 0.6115 - accuracy: 0.6686 - val_loss: 0.6155 - val_accuracy: 0.6640
Epoch 6/10
1847/1847 [==============================] - 55s 30ms/step - loss: 0.6047 - accuracy: 0.6719 - val_loss: 0.6148 - val_accuracy: 0.6630
Epoch 7/10
1847/1847 [==============================] - 54s 29ms/step - loss: 0.5994 - accuracy: 0.67

100%|██████████| 100/100 [00:01<00:00, 52.47it/s]


Calculating float32...
Epoch 1/10
1847/1847 [==============================] - 59s 30ms/step - loss: 0.7674 - accuracy: 0.6379 - val_loss: 0.6687 - val_accuracy: 0.6457
Epoch 2/10
1847/1847 [==============================] - 55s 30ms/step - loss: 0.6495 - accuracy: 0.6555 - val_loss: 0.6527 - val_accuracy: 0.6462
Epoch 3/10
1847/1847 [==============================] - 55s 30ms/step - loss: 0.6338 - accuracy: 0.6602 - val_loss: 0.6319 - val_accuracy: 0.6597
Epoch 4/10
1847/1847 [==============================] - 54s 29ms/step - loss: 0.6211 - accuracy: 0.6641 - val_loss: 0.6352 - val_accuracy: 0.6509
Epoch 5/10
1847/1847 [==============================] - 55s 30ms/step - loss: 0.6113 - accuracy: 0.6678 - val_loss: 0.6173 - val_accuracy: 0.6621
Epoch 6/10
1847/1847 [==============================] - 55s 30ms/step - loss: 0.6044 - accuracy: 0.6701 - val_loss: 0.6716 - val_accuracy: 0.6552
Epoch 7/10
1847/1847 [==============================] - 55s 30ms/step - loss: 0.5995 - accuracy: 0.67

100%|██████████| 100/100 [00:01<00:00, 52.94it/s]


Calculating float32...
Epoch 1/10
1847/1847 [==============================] - 58s 30ms/step - loss: 0.7660 - accuracy: 0.6357 - val_loss: 0.6686 - val_accuracy: 0.6492
Epoch 2/10
1847/1847 [==============================] - 55s 30ms/step - loss: 0.6524 - accuracy: 0.6554 - val_loss: 8.3830 - val_accuracy: 0.4982
Epoch 3/10
1847/1847 [==============================] - 54s 29ms/step - loss: 0.6404 - accuracy: 0.6581 - val_loss: 0.6980 - val_accuracy: 0.6402
Epoch 4/10
1847/1847 [==============================] - 54s 29ms/step - loss: 0.6266 - accuracy: 0.6621 - val_loss: 0.6511 - val_accuracy: 0.6524
Epoch 5/10
1847/1847 [==============================] - 55s 30ms/step - loss: 0.6161 - accuracy: 0.6654 - val_loss: 0.6284 - val_accuracy: 0.6527
Epoch 6/10
1847/1847 [==============================] - 55s 30ms/step - loss: 0.6079 - accuracy: 0.6686 - val_loss: 0.6188 - val_accuracy: 0.6584
Epoch 7/10
1847/1847 [==============================] - 55s 30ms/step - loss: 0.6022 - accuracy: 0.67

In [26]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, callbacks, optimizers
from tensorflow.keras.regularizers import l2
import gc
import os
INPUT_SIZE = 52
PATH_MODELS =os.path.join(PATH_MODELS_PARENT,f"models_seq_{INPUT_SIZE}")
os.makedirs(PATH_MODELS, exist_ok=True)
pinwheel_sizes = [26]

cumulative_sizes = np.cumsum([0] + pinwheel_sizes)


model_accuracies = []

# Loop over each sequence length
seq_len = INPUT_SIZE
# Iterate over each wheel
for wheel, (start_pin, end_pin) in enumerate(zip(cumulative_sizes, cumulative_sizes[1:])):
    train_all_pins = False  # Flag to determine if all pins should be trained

    # Iterate over each pin in the current wheel
    for pin in range(20, end_pin):
        while True:
            # Extract the target values for the current pin
            x, y = load_partial_data(100,10000)

            targets = y[:, pin]
            # Split the data into training and testing sets
            X_train, X_test, y_train, y_test = train_test_split(x, targets, test_size=0.2, random_state=17)

            # Create and compile the ResNet model
            model_pin = make_resnet(seq_len, num_filters=52, depth=5, num_outputs=1)
            model_pin.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])

            # Setup callbacks for model saving and learning rate scheduling
            checkpoint_cb_pin = callbacks.ModelCheckpoint(PATH_MODELS+f'/best_model_wheel_{wheel}_pin_{pin}.h5', save_best_only=True)
            lr_scheduler_cb_pin = callbacks.LearningRateScheduler(lambda epoch: 1e-3 * 0.95 ** epoch)

            # Train the model
            history_pin = model_pin.fit(X_train, y_train, batch_size=520, epochs=10, validation_split=0.2, callbacks=[checkpoint_cb_pin, lr_scheduler_cb_pin])
            # Evaluate the model on the test set
            test_loss_pin, test_accuracy_pin = model_pin.evaluate(X_test, y_test)
            print(f"Seq Length {seq_len}, Wheel {wheel}, Pin {pin}: Test Loss: {test_loss_pin}, Test Accuracy: {test_accuracy_pin}")

            # Check if the current pin is the first pin
            if pin == start_pin:
                # If the accuracy is higher than 0.6, set the flag to train all pins
                if test_accuracy_pin > 0.67:
                    train_all_pins = True
                else:
                    break  # Skip training the remaining pins for this wheel
            #retraining for pins vs low accuracy        
            if test_accuracy_pin > 0.77:

                # Record the model's accuracy if it's the first pin or if all pins are being trained
                if train_all_pins or pin == start_pin:
                    model_accuracies.append({
                        'Sequence Length': seq_len,
                        'Wheel Number': wheel,
                        'Pin Number': pin,
                        'Accuracy': test_accuracy_pin
                    })

                    # Save the final model
                    model_pin.save(PATH_MODELS+f'/best_model_wheel_{wheel}_pin_{pin}.h5')

                # Clear the TensorFlow session and collect garbage to free memory
                tf.keras.backend.clear_session()
                gc.collect()
                break

            
# Convert the list of accuracies to a DataFrame
accuracy_df = pd.DataFrame(model_accuracies)

# Save the DataFrame to an Excel file
accuracy_df.to_excel('model_accuracies.xlsx', index=False)

100%|██████████| 100/100 [00:01<00:00, 53.39it/s]


Calculating float32...
Epoch 1/10
1231/1231 [==============================] - 63s 48ms/step - loss: 0.7709 - accuracy: 0.7161 - val_loss: 0.6618 - val_accuracy: 0.7111
Epoch 2/10
1231/1231 [==============================] - 59s 48ms/step - loss: 0.5861 - accuracy: 0.7468 - val_loss: 0.5915 - val_accuracy: 0.7319
Epoch 3/10
1231/1231 [==============================] - 59s 48ms/step - loss: 0.5550 - accuracy: 0.7541 - val_loss: 0.5682 - val_accuracy: 0.7419
Epoch 4/10
1231/1231 [==============================] - 60s 48ms/step - loss: 0.5347 - accuracy: 0.7635 - val_loss: 0.5607 - val_accuracy: 0.7452
Epoch 5/10
1231/1231 [==============================] - 60s 49ms/step - loss: 0.5168 - accuracy: 0.7737 - val_loss: 0.5852 - val_accuracy: 0.7271
Epoch 6/10
1231/1231 [==============================] - 60s 49ms/step - loss: 0.5020 - accuracy: 0.7842 - val_loss: 0.5510 - val_accuracy: 0.7486
Epoch 7/10
1231/1231 [==============================] - 60s 49ms/step - loss: 0.4874 - accuracy: 0.79

100%|██████████| 100/100 [00:02<00:00, 49.39it/s]


Calculating float32...
Epoch 1/10
1231/1231 [==============================] - 61s 47ms/step - loss: 0.7708 - accuracy: 0.7148 - val_loss: 0.6983 - val_accuracy: 0.6797
Epoch 2/10
1231/1231 [==============================] - 59s 48ms/step - loss: 0.5846 - accuracy: 0.7484 - val_loss: 0.6128 - val_accuracy: 0.7386
Epoch 3/10
1231/1231 [==============================] - 59s 48ms/step - loss: 0.5547 - accuracy: 0.7545 - val_loss: 0.5583 - val_accuracy: 0.7484
Epoch 4/10
1231/1231 [==============================] - 60s 48ms/step - loss: 0.5330 - accuracy: 0.7630 - val_loss: 0.5423 - val_accuracy: 0.7571
Epoch 5/10
1231/1231 [==============================] - 60s 49ms/step - loss: 0.5150 - accuracy: 0.7740 - val_loss: 0.6032 - val_accuracy: 0.7331
Epoch 6/10
1231/1231 [==============================] - 60s 49ms/step - loss: 0.4975 - accuracy: 0.7857 - val_loss: 0.5556 - val_accuracy: 0.7660
Epoch 7/10
1231/1231 [==============================] - 60s 49ms/step - loss: 0.4814 - accuracy: 0.79

100%|██████████| 100/100 [00:01<00:00, 52.87it/s]


...

Calculating float32...
Epoch 1/10
1231/1231 [==============================] - 61s 46ms/step - loss: 0.7667 - accuracy: 0.7171 - val_loss: 0.6768 - val_accuracy: 0.7232
Epoch 2/10
1231/1231 [==============================] - 59s 48ms/step - loss: 0.5852 - accuracy: 0.7477 - val_loss: 0.5769 - val_accuracy: 0.7398
Epoch 3/10
1231/1231 [==============================] - 60s 48ms/step - loss: 0.5522 - accuracy: 0.7556 - val_loss: 0.5774 - val_accuracy: 0.7429
Epoch 4/10
1231/1231 [==============================] - 60s 49ms/step - loss: 0.5306 - accuracy: 0.7646 - val_loss: 0.5737 - val_accuracy: 0.7363
Epoch 5/10
1231/1231 [==============================] - 60s 49ms/step - loss: 0.5126 - accuracy: 0.7752 - val_loss: 0.5525 - val_accuracy: 0.7543
Epoch 6/10
1231/1231 [==============================] - 60s 48ms/step - loss: 0.4972 - accuracy: 0.7863 - val_loss: 0.6041 - val_accuracy: 0.7173
Epoch 7/10
1231/1231 [==============================] - 60s 49ms/step - loss: 0.4824 - accuracy: 0.79

100%|██████████| 100/100 [00:03<00:00, 31.74it/s]


Calculating float32...
Epoch 1/10
1231/1231 [==============================] - 80s 60ms/step - loss: 0.7780 - accuracy: 0.7166 - val_loss: 0.6804 - val_accuracy: 0.6913
Epoch 2/10
1231/1231 [==============================] - 77s 62ms/step - loss: 0.5864 - accuracy: 0.7480 - val_loss: 0.5795 - val_accuracy: 0.7470
Epoch 3/10
1231/1231 [==============================] - 69s 56ms/step - loss: 0.5529 - accuracy: 0.7549 - val_loss: 0.5729 - val_accuracy: 0.7399
Epoch 4/10
1231/1231 [==============================] - 77s 63ms/step - loss: 0.5310 - accuracy: 0.7656 - val_loss: 0.5423 - val_accuracy: 0.7565
Epoch 5/10
1231/1231 [==============================] - 76s 61ms/step - loss: 0.5127 - accuracy: 0.7769 - val_loss: 0.6587 - val_accuracy: 0.7314
Epoch 6/10
1231/1231 [==============================] - 74s 60ms/step - loss: 0.4962 - accuracy: 0.7872 - val_loss: 0.5507 - val_accuracy: 0.7580
Epoch 7/10
1231/1231 [==============================] - 59s 48ms/step - loss: 0.4816 - accuracy: 0.79

100%|██████████| 100/100 [00:03<00:00, 26.54it/s]


Calculating float32...
Epoch 1/10
1231/1231 [==============================] - 80s 60ms/step - loss: 0.7662 - accuracy: 0.7184 - val_loss: 0.6331 - val_accuracy: 0.7291
Epoch 2/10
1231/1231 [==============================] - 76s 62ms/step - loss: 0.5829 - accuracy: 0.7482 - val_loss: 0.5836 - val_accuracy: 0.7408
Epoch 3/10
1231/1231 [==============================] - 76s 62ms/step - loss: 0.5504 - accuracy: 0.7557 - val_loss: 0.5653 - val_accuracy: 0.7429
Epoch 4/10
1231/1231 [==============================] - 75s 61ms/step - loss: 0.5296 - accuracy: 0.7659 - val_loss: 0.8482 - val_accuracy: 0.6422
Epoch 5/10
1231/1231 [==============================] - 77s 62ms/step - loss: 0.5123 - accuracy: 0.7770 - val_loss: 0.5587 - val_accuracy: 0.7598
Epoch 6/10
1231/1231 [==============================] - 75s 61ms/step - loss: 0.4952 - accuracy: 0.7877 - val_loss: 0.5761 - val_accuracy: 0.7421
Epoch 7/10
1231/1231 [==============================] - 77s 62ms/step - loss: 0.4789 - accuracy: 0.79

In [ ]:
# 